In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns
%matplotlib inline


import pysal as ps
import libpysal
import spreg

import pysal.explore
import pysal.model
from libpysal import weights

import statsmodels.formula.api as smf
import statsmodels.api as sm

import statistics
from itertools import combinations

tfont = {'fontname':'Liberation Sans Narrow', 'horizontalalignment':'left'}
from matplotlib import rcParams
rcParams['font.family'] = 'Liberation Sans Narrow'

/opt/conda/lib/python3.7/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [2]:
sns.set_style({'font.family':'serif', 'font.serif':['Liberation Sans Narrow']})

In [3]:
sns.set(font="Liberation Sans Narrow")

In [4]:
import matplotlib.style as style
style.use('fivethirtyeight')
# style.use('ggplot')

In [5]:
msoa_cropped = gpd.read_file('../data/output.msoa_cropped_v2.geojson')
msoa_cropped = msoa_cropped[['MSOA11CD', 'MSOA11NM','geometry']]

In [6]:
bus_msoa = pd.read_csv('../data/y/bus_msoa.csv')
bus_msoa_gdf = msoa_cropped.merge(bus_msoa, how='left')

In [7]:
c_arr_msoa = pd.read_csv('../data/y/c_arr_msoa.csv')
c_arr_msoa_gdf = msoa_cropped.merge(c_arr_msoa, how='left')

In [8]:
c_dur_msoa = pd.read_csv('../data/y/c_dur_msoa.csv')
c_dur_msoa_gdf = msoa_cropped.merge(c_dur_msoa, how='left')

In [9]:
msoa_final = pd.read_csv('../data/final_df_msoa.csv')

In [10]:
msoa_final.head(2)

,MSOA11CD,population,employment,deprivation,street_length,median_ebc,msoa_area,street_density,green_area,tourism_density,hs_building_area,bus_stops_density,bike_stations_density
0,E02000001,8008.50,356706,14.8050,140254.538,34574.0,2.905399e+06,4.827376,55339.928903,27.0,328917.680468,109,43
1,E02000180,9373.87,2218,24.2006,13919.782,4252.0,4.856208e+05,0.479100,15065.659009,0.0,7759.359406,19,1


In [11]:
msoa_to_categories202103 = pd.read_csv('../data/x/msoa_to_categories202103.csv')
msoa_to_categories202003 = pd.read_csv('../data/x/msoa_to_categories202003.csv')
msoa_to_categories201903 = pd.read_csv('../data/x/msoa_to_categories201903.csv')

In [12]:
msoa_to_categories201903.columns = ['MSOA11CD', 'total19', 'office19', 'retail19', 'leisure19', 'industrial19', 
                                    'other19','ret_restaurants19', 'ret_hs19', 'ret_superstores19']
msoa_to_categories202003.columns = ['MSOA11CD', 'total20', 'office20', 'retail20', 'leisure20', 'industrial20',
                                    'other20', 'ret_restaurants20', 'ret_hs20', 'ret_superstores20']
msoa_to_categories202103.columns = ['MSOA11CD', 'total21', 'office21', 'retail21', 'leisure21', 'industrial21',
                                    'other21', 'ret_restaurants21', 'ret_hs21', 'ret_superstores21']

In [13]:
# gdf = bus_msoa_gdf.merge(c_arr_msoa_gdf).merge(msoa_final).merge(msoa_to_categories201903)\
#     .merge(msoa_to_categories202003).merge(msoa_to_categories202103)

In [14]:
#2019
gdf = bus_msoa_gdf.merge(c_arr_msoa_gdf).merge(msoa_final).merge(msoa_to_categories201903) \
    .merge(msoa_to_categories202003).merge(msoa_to_categories202103)

In [15]:
gdf = gdf.merge(c_dur_msoa_gdf)

In [16]:
gdf = gdf.fillna(0)

In [17]:
gdf.columns

Index(['MSOA11CD', 'MSOA11NM', 'geometry', 'b_passengers201906',
       'b_passengers202006', 'b_passengers202106', 'c_trips201906',
       'c_trips202006', 'c_trips202106', 'population', 'employment',
       'deprivation', 'street_length', 'median_ebc', 'msoa_area',
       'street_density', 'green_area', 'tourism_density', 'hs_building_area',
       'bus_stops_density', 'bike_stations_density', 'total19', 'office19',
       'retail19', 'leisure19', 'industrial19', 'other19', 'ret_restaurants19',
       'ret_hs19', 'ret_superstores19', 'total20', 'office20', 'retail20',
       'leisure20', 'industrial20', 'other20', 'ret_restaurants20', 'ret_hs20',
       'ret_superstores20', 'total21', 'office21', 'retail21', 'leisure21',
       'industrial21', 'other21', 'ret_restaurants21', 'ret_hs21',
       'ret_superstores21', 'c_dur201906', 'c_dur202006', 'c_dur202106'],
      dtype='object')

In [18]:
c_dur_msoa_gdf.columns

Index(['MSOA11CD', 'MSOA11NM', 'geometry', 'c_dur201906', 'c_dur202006',
       'c_dur202106'],
      dtype='object')

### Descriptive stats

In [19]:
columns = ['MSOA11CD', 'b_passengers201906',
       'b_passengers202006', 'b_passengers202106', 'c_trips201906',
       'c_trips202006', 'c_trips202106', 'population', 'employment',
       'deprivation', 'bus_stops_density', 'bike_stations_density',
           'street_length', 'median_ebc', 'msoa_area',
       'green_area', 'tourism_density', 'hs_building_area',
       'total19', 'office19', 'retail19', 'leisure19', 'industrial19',
       'other19', 'ret_restaurants19', 'ret_hs19', 'ret_superstores19',
        'total20', 'office20', 'retail20', 'leisure20', 'industrial20',
        'other20', 'ret_restaurants20', 'ret_hs20', 'ret_superstores20',
        'total21', 'office21', 'retail21', 'leisure21', 'industrial21',
         'other21', 'ret_restaurants21', 'ret_hs21', 'ret_superstores21',
           'c_dur201906', 'c_dur202006',
       'c_dur202106'
       ]

df = gdf[columns]
df = df.set_index('MSOA11CD')
columns.pop(0)

var_min = []
var_max = []
var_mean = []
var_stdev = []

for col in df:
    var_min.append(round(df[col].min(), 2))
    var_max.append(round(df[col].max(), 2))
    var_mean.append(round(df[col].mean(), 2))
    var_stdev.append(round(statistics.stdev(df[col]),2))

In [20]:
d_stat = pd.DataFrame(list(zip(columns, var_min, var_max, var_mean, var_stdev)), 
                      columns=(['var','min','max','mean','stdev']))

In [21]:
# fig, axes = plt.subplots(len(df.columns)//4+1, 4, figsize=(12, 36))

# i = 0
# for triaxis in axes:
#     for axis in triaxis:
#         df.hist(column = df.columns[i], bins = 50, ax=axis)
#         axis.title.set_font('Liberation Sans Narrow')
#         i = i+1

In [22]:
df_log = df.applymap(lambda x: np.log(x+1))
# rename columns
df_log.columns = 'log_' + df_log.columns

In [23]:
# fig, axes = plt.subplots(len(df_log.columns)//4+1, 4, figsize=(12, 36))

# i = 0
# for triaxis in axes:
#     for axis in triaxis:
#         df_log.hist(column = df_log.columns[i], bins = 50, ax=axis)
#         axis.title.set_font('Liberation Sans Narrow')
#         i = i+1

### 1. OLS

In [24]:
data2019 = df_log[['log_b_passengers201906', 'log_c_trips201906', 'log_population', 'log_employment',
       'log_street_length', 'log_median_ebc','log_msoa_area','log_green_area',
       'log_tourism_density', 'log_hs_building_area', 'log_total19',
       'log_office19', 'log_retail19', 'log_leisure19', 'log_industrial19',
       'log_other19', 'log_ret_restaurants19', 'log_ret_hs19',
       'log_ret_superstores19']].merge(df[['deprivation']], left_index=True, right_index=True)

In [25]:
# plt.rcParams["axes.grid"] = False
# f = plt.figure(figsize=(16, 16))
# plt.matshow(data2019.corr(), cmap='bwr', fignum=f.number)
# plt.xticks(range(data2019.shape[1]), data2019.columns, fontsize=14, rotation=45)
# plt.yticks(range(data2019.shape[1]), data2019.columns, fontsize=14)
# cb = plt.colorbar()
# cb.ax.tick_params(labelsize=12)
# plt.title('Correlation Matrix', fontsize=16);

In [26]:
corr = data2019.corr()
mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True
corr[mask] = np.nan

(corr
 .style
 .background_gradient(cmap=sns.diverging_palette(220, 10, as_cmap=True), axis=None, vmin=-1, vmax=1)
 .highlight_null(null_color='#f1f1f1')  # Color NaNs grey
 .set_precision(2))

,log_b_passengers201906,log_c_trips201906,log_population,log_employment,log_street_length,log_median_ebc,log_msoa_area,log_green_area,log_tourism_density,log_hs_building_area,log_total19,log_office19,log_retail19,log_leisure19,log_industrial19,log_other19,log_ret_restaurants19,log_ret_hs19,log_ret_superstores19,deprivation
log_b_passengers201906,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
log_c_trips201906,0.45,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
log_population,0.27,0.04,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
log_employment,0.65,0.61,0.21,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
log_street_length,0.61,0.41,0.46,0.60,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
log_median_ebc,0.52,0.50,0.34,0.60,0.64,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
log_msoa_area,0.59,0.34,0.46,0.54,0.94,0.53,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
log_green_area,0.12,0.01,0.15,-0.00,0.43,0.13,0.45,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
log_tourism_density,0.49,0.58,0.14,0.66,0.54,0.58,0.49,0.12,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
log_hs_building_area,0.38,0.26,-0.11,0.29,0.04,0.28,0.06,-0.08,0.24,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [27]:
from statsmodels.stats.outliers_influence import variance_inflation_factor 
from statsmodels.tools.tools import add_constant

def drop_column_using_vif_(df, thresh):
    while True:
        # adding a constatnt item to the data
        df_with_const = add_constant(df)

        vif_df = pd.Series([variance_inflation_factor(df_with_const.values, i) 
               for i in range(df_with_const.shape[1])], name= "VIF",
              index=df_with_const.columns).to_frame()

        # drop the const
        vif_df = vif_df.drop('const')
        
        print('Max VIF:', vif_df.VIF.max())
        
        # if the largest VIF is above the thresh, remove a variable with the largest VIF
        if vif_df.VIF.max() > thresh:
            # If there are multiple variables with the maximum VIF, choose the first one
            index_to_drop = vif_df.index[vif_df.VIF == vif_df.VIF.max()].tolist()[0]
            print('Dropping: {}'.format(index_to_drop))
            df = df.drop(columns = index_to_drop)
        else:
            # No VIF is above threshold. Exit the loop
            break

    return df

In [28]:
data2019_x = data2019.drop(columns=['log_c_trips201906','log_b_passengers201906'], axis=1)
data2019_y = data2019['log_b_passengers201906']

In [29]:
data2019_x.shape

(154, 18)

In [30]:
df_predictors_select_VIF = drop_column_using_vif_(data2019_x, 5)
print("The columns remaining after VIF selection are:")
print(df_predictors_select_VIF.columns)

Max VIF: 686.716527787498
Dropping: log_retail19
Max VIF: 73.68119717402045
Dropping: log_total19
Max VIF: 12.325476379784385
Dropping: log_street_length
Max VIF: 8.226062384623742
Dropping: log_employment
Max VIF: 7.363324103040719
Dropping: log_ret_restaurants19
Max VIF: 5.328786698136519
Dropping: log_other19
Max VIF: 3.4205103141974194
The columns remaining after VIF selection are:
Index(['log_population', 'log_median_ebc', 'log_msoa_area', 'log_green_area',
       'log_tourism_density', 'log_hs_building_area', 'log_office19',
       'log_leisure19', 'log_industrial19', 'log_ret_hs19',
       'log_ret_superstores19', 'deprivation'],
      dtype='object')


In [31]:
ols_formula_2019 = 'log_b_passengers201906 ~ log_population + log_median_ebc + log_msoa_area + \
                log_green_area + log_tourism_density + log_hs_building_area + log_office19 +\
                log_leisure19 + log_industrial19 + log_ret_hs19 + log_ret_superstores19 + deprivation'

initial_model = sm.formula.ols(ols_formula_2019,
                               data2019).fit()
initial_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                              
==================================================================================
Dep. Variable:     log_b_passengers201906   R-squared:                       0.623
Model:                                OLS   Adj. R-squared:                  0.590
Method:                     Least Squares   F-statistic:                     19.38
Date:                    Mon, 13 Sep 2021   Prob (F-statistic):           2.56e-24
Time:                            14:21:26   Log-Likelihood:                -123.72
No. Observations:                     154   AIC:                             273.4
Df Residuals:                         141   BIC:                             312.9
Df Model:                              12                                         
Covariance Type:                nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                -6.4582      2.284     -2.828      0.005     -10.974      -1.943
log_population           -0.0028      0.236     -0.012      0.991      -0.470       0.464
log_median_ebc           -0.0429      0.125     -0.344      0.731      -0.289       0.203
log_msoa_area             0.9700      0.168      5.775      0.000       0.638       1.302
log_green_area           -0.0312      0.025     -1.225      0.223      -0.082       0.019
log_tourism_density      -0.0043      0.071     -0.060      0.952      -0.145       0.136
log_hs_building_area      0.0437      0.019      2.285      0.024       0.006       0.081
log_office19              0.0906      0.057      1.591      0.114      -0.022       0.203
log_leisure19             0.0125      0.069      0.182      0.856      -0.123       0.148
log_industrial19         -0.0542      0.048     -1.125      0.262      -0.149       0.041
log_ret_hs19              0.3505      0.096      3.635      0.000       0.160       0.541
log_ret_superstores19     0.0039      0.062      0.063      0.950      -0.119       0.127
deprivation               0.0179      0.006      2.858      0.005       0.006       0.030
==============================================================================
Omnibus:                        1.129   Durbin-Watson:                   2.023
Prob(Omnibus):                  0.569   Jarque-Bera (JB):                0.800
Skew:                           0.158   Prob(JB):                        0.670
Kurtosis:                       3.156   Cond. No.                     1.74e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.74e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### PYSAL

In [32]:
from libpysal.weights import Queen
wq = Queen.from_dataframe(gdf)
wq.transform = 'R'

In [33]:
gdf = gdf.set_index('MSOA11CD')

In [34]:
gdf2019 = gdf.loc[:, ['geometry']]
gdf2019 = gdf2019.merge(data2019, left_index=True, right_index=True)

In [35]:
gdf2019.columns

Index(['geometry', 'log_b_passengers201906', 'log_c_trips201906',
       'log_population', 'log_employment', 'log_street_length',
       'log_median_ebc', 'log_msoa_area', 'log_green_area',
       'log_tourism_density', 'log_hs_building_area', 'log_total19',
       'log_office19', 'log_retail19', 'log_leisure19', 'log_industrial19',
       'log_other19', 'log_ret_restaurants19', 'log_ret_hs19',
       'log_ret_superstores19', 'deprivation'],
      dtype='object')

In [36]:
data2019_x = gdf2019.drop(columns=['log_b_passengers201906','log_c_trips201906','geometry'], axis=1)
data2019_y = gdf2019['log_b_passengers201906']

In [37]:
data2019_x.columns

Index(['log_population', 'log_employment', 'log_street_length',
       'log_median_ebc', 'log_msoa_area', 'log_green_area',
       'log_tourism_density', 'log_hs_building_area', 'log_total19',
       'log_office19', 'log_retail19', 'log_leisure19', 'log_industrial19',
       'log_other19', 'log_ret_restaurants19', 'log_ret_hs19',
       'log_ret_superstores19', 'deprivation'],
      dtype='object')

In [38]:
variable_names = ['log_population', 'log_employment', 'log_street_length',
       'log_median_ebc', 'log_msoa_area', 'log_green_area',
       'log_tourism_density', 'log_hs_building_area', 'log_total19',
       'log_office19', 'log_retail19', 'log_leisure19', 'log_industrial19',
       'log_other19', 'log_ret_restaurants19', 'log_ret_hs19',
       'log_ret_superstores19', 'deprivation']

In [39]:
variable_names_after_vif = ['log_population', 'log_median_ebc', 'log_msoa_area', 'log_green_area',
       'log_tourism_density', 'log_hs_building_area', 'log_office19',
       'log_leisure19', 'log_industrial19', 'log_ret_hs19',
       'log_ret_superstores19', 'deprivation']

In [40]:
m1 = pysal.model.spreg.OLS(gdf2019[['log_b_passengers201906']].values,
               gdf2019[variable_names].values,
               name_y='log_b_passengers201906',
               name_x=variable_names)
print(m1.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :        None
Dependent Variable  :log_b_passengers201906                Number of Observations:         154
Mean dependent var  :      8.6244                Number of Variables   :          19
S.D. dependent var  :      0.8824                Degrees of Freedom    :         135
R-squared           :      0.6658
Adjusted R-squared  :      0.6212
Sum squared residual:      39.816                F-statistic           :     14.9392
Sigma-square        :       0.295                Prob(F-statistic)     :   5.465e-24
S.E. of regression  :       0.543                Log likelihood        :    -114.359
Sigma-square ML     :       0.259                Akaike info criterion :     266.718
S.E of regression ML:      0.5085                Schwarz criterion     :     324.420

-------------------------------------------------------------------

### Spatial Lag Model

In [41]:
m2 = pysal.model.spreg.ML_Lag(gdf2019[['log_b_passengers201906']].values,
               gdf2019[variable_names].values,
               name_y='log_b_passengers201906',
               name_x=variable_names,
                  w=wq)


print(m2.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL LAG (METHOD = FULL)
-----------------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :log_b_passengers201906                Number of Observations:         154
Mean dependent var  :      8.6244                Number of Variables   :          20
S.D. dependent var  :      0.8824                Degrees of Freedom    :         134
Pseudo R-squared    :      0.6667
Spatial Pseudo R-squared:  0.6647
Sigma-square ML     :       0.258                Log likelihood        :    -114.175
S.E of regression   :       0.508                Akaike info criterion :     268.351
                                                 Schwarz criterion     :     329.090

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
-------------------

### Spatial Error Model

In [42]:
m3 = pysal.model.spreg.ML_Error(gdf2019[['log_b_passengers201906']].values,
               gdf2019[variable_names].values,
               name_y='log_b_passengers201906',
               name_x=variable_names,
                  w=wq)
print(m3.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL ERROR (METHOD = FULL)
-------------------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :log_b_passengers201906                Number of Observations:         154
Mean dependent var  :      8.6244                Number of Variables   :          19
S.D. dependent var  :      0.8824                Degrees of Freedom    :         135
Pseudo R-squared    :      0.6640
Sigma-square ML     :       0.253                Log likelihood        :    -113.299
S.E of regression   :       0.503                Akaike info criterion :     264.599
                                                 Schwarz criterion     :     322.301

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
-------------------------------------------------

In [43]:
from sklearn.metrics import mean_squared_error as mse

y = gdf2019[['log_b_passengers201906']]

mses = pd.Series({'OLS': mse(y, m1.predy.flatten()), \
                     'SLM': mse(y, m2.predy.flatten()), \
                     'SEM': mse(y, m3.predy.flatten())
                    })
mses.sort_values()

SLM    0.257787
OLS    0.258543
SEM    0.259952
dtype: float64

## Final SLM models

In [44]:
data2019 = df_log[['log_c_dur201906', 'log_b_passengers201906', 'log_c_trips201906', 'log_population', 'log_employment',
       'log_bus_stops_density', 'log_bike_stations_density',
       'log_street_length', 'log_median_ebc','log_msoa_area','log_green_area',
       'log_tourism_density', 'log_hs_building_area', 'log_total19',
       'log_office19', 'log_retail19', 'log_leisure19', 'log_industrial19',
       'log_other19', 'log_ret_restaurants19', 'log_ret_hs19',
       'log_ret_superstores19']].merge(df[['deprivation']], left_index=True, right_index=True)

data2020 = df_log[['log_c_dur202006', 'log_b_passengers202006', 'log_c_trips202006', 'log_population', 'log_employment',
       'log_bus_stops_density', 'log_bike_stations_density',
       'log_street_length', 'log_median_ebc','log_msoa_area','log_green_area',
       'log_tourism_density', 'log_hs_building_area', 'log_total20', 
       'log_office20','log_retail20', 'log_leisure20', 'log_industrial20', 
       'log_other20', 'log_ret_restaurants20', 'log_ret_hs20', 
       'log_ret_superstores20']].merge(df[['deprivation']], left_index=True, right_index=True)

data2021 = df_log[['log_c_dur202106', 'log_b_passengers202106', 'log_c_trips202106', 'log_population', 'log_employment',
       'log_bus_stops_density', 'log_bike_stations_density',
       'log_street_length', 'log_median_ebc','log_msoa_area','log_green_area',
       'log_tourism_density', 'log_hs_building_area', 'log_total21', 
       'log_office21', 'log_retail21', 'log_leisure21', 'log_industrial21', 
       'log_other21', 'log_ret_restaurants21', 'log_ret_hs21', 
       'log_ret_superstores21']].merge(df[['deprivation']], left_index=True, right_index=True)

In [45]:
gdf2019 = gdf.loc[:, ['geometry']]
gdf2019 = gdf2019.merge(data2019, left_index=True, right_index=True)

gdf2020 = gdf.loc[:, ['geometry']]
gdf2020 = gdf2020.merge(data2020, left_index=True, right_index=True)

gdf2021 = gdf.loc[:, ['geometry']]
gdf2021 = gdf2021.merge(data2021, left_index=True, right_index=True)

In [46]:
variable_names19 = ['log_population', 'log_employment', 'deprivation',
        'log_bus_stops_density', 'log_bike_stations_density',
        'log_street_length','log_median_ebc', 'log_msoa_area', 'log_green_area',
       'log_tourism_density', 'log_hs_building_area', 'log_total19',
       'log_office19', 'log_retail19', 'log_leisure19', 'log_industrial19',
       'log_other19', 'log_ret_restaurants19', 'log_ret_hs19',
       'log_ret_superstores19']

variable_names20 = ['log_population', 'log_employment', 'deprivation',
        'log_bus_stops_density', 'log_bike_stations_density',
        'log_street_length','log_median_ebc', 'log_msoa_area', 'log_green_area',
       'log_tourism_density', 'log_hs_building_area', 'log_total20',
       'log_office20', 'log_retail20', 'log_leisure20', 'log_industrial20',
       'log_other20', 'log_ret_restaurants20', 'log_ret_hs20',
       'log_ret_superstores20']

variable_names21 = ['log_population', 'log_employment', 'deprivation',
        'log_bus_stops_density', 'log_bike_stations_density',
        'log_street_length','log_median_ebc', 'log_msoa_area', 'log_green_area',
       'log_tourism_density', 'log_hs_building_area', 'log_total21',
       'log_office21', 'log_retail21', 'log_leisure21', 'log_industrial21',
       'log_other21', 'log_ret_restaurants21', 'log_ret_hs21',
       'log_ret_superstores21']

In [47]:
m2 = pysal.model.spreg.ML_Lag(gdf2019[['log_b_passengers201906']].values,
                  gdf2019[variable_names19].values,
                  w=wq,
                  name_y='log_b_passengers201906',
                  name_x=variable_names19)
print(m2.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL LAG (METHOD = FULL)
-----------------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :log_b_passengers201906                Number of Observations:         154
Mean dependent var  :      8.6244                Number of Variables   :          22
S.D. dependent var  :      0.8824                Degrees of Freedom    :         132
Pseudo R-squared    :      0.7402
Spatial Pseudo R-squared:  0.7394
Sigma-square ML     :       0.201                Log likelihood        :     -94.959
S.E of regression   :       0.448                Akaike info criterion :     233.918
                                                 Schwarz criterion     :     300.731

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
-------------------

In [48]:
df_res = pd.DataFrame()
df_res['variable'] = m2.name_x

pr2 = []
aic = []
logll = []

In [49]:
colname_beta = m2.name_y + '_beta'
df_res[colname_beta] = m2.betas
df_res[colname_beta] = np.round(df_res[colname_beta], 3)
df_res[colname_beta] = df_res[colname_beta].astype(str)

colname_se = m2.name_y + '_se'
df_res[colname_se] = m2.std_err
df_res[colname_se] = np.round(df_res[colname_se], 3)

colname_p = m2.name_y + '_p'
df_res[colname_p] = [t[1] for t in  m2.z_stat]
df_res[colname_p] = np.round(df_res[colname_p], 3)

df_res.loc[df_res[colname_p] <= 0.05, colname_beta] = df_res[colname_beta] + "*"

pr2.append(m2.pr2)
aic.append(m2.aic)
logll.append(m2.logll)

In [50]:
m2 = pysal.model.spreg.ML_Lag(gdf2019[['log_c_trips201906']].values,
                  gdf2019[variable_names19].values,
                  w=wq,
                  name_y='log_c_trips201906',
                  name_x=variable_names19)
print(m2.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL LAG (METHOD = FULL)
-----------------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :log_c_trips201906                Number of Observations:         154
Mean dependent var  :      4.7863                Number of Variables   :          22
S.D. dependent var  :      1.1372                Degrees of Freedom    :         132
Pseudo R-squared    :      0.7325
Spatial Pseudo R-squared:  0.7328
Sigma-square ML     :       0.344                Log likelihood        :    -136.292
S.E of regression   :       0.586                Akaike info criterion :     316.584
                                                 Schwarz criterion     :     383.397

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------

In [51]:
colname_beta = m2.name_y + '_beta'
df_res[colname_beta] = m2.betas
df_res[colname_beta] = np.round(df_res[colname_beta], 3)
df_res[colname_beta] = df_res[colname_beta].astype(str)

colname_se = m2.name_y + '_se'
df_res[colname_se] = m2.std_err
df_res[colname_se] = np.round(df_res[colname_se], 3)

colname_p = m2.name_y + '_p'
df_res[colname_p] = [t[1] for t in  m2.z_stat]
df_res[colname_p] = np.round(df_res[colname_p], 3)

df_res.loc[df_res[colname_p] <= 0.05, colname_beta] = df_res[colname_beta] + "*"

pr2.append(m2.pr2)
aic.append(m2.aic)
logll.append(m2.logll)

In [52]:
m2 = pysal.model.spreg.ML_Lag(gdf2020[['log_b_passengers202006']].values,
                  gdf2020[variable_names20].values,
                  w=wq,
                  name_y='log_b_passengers202006',
                  name_x=variable_names)
print(m2.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL LAG (METHOD = FULL)
-----------------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :log_b_passengers202006                Number of Observations:         154
Mean dependent var  :      7.3406                Number of Variables   :          22
S.D. dependent var  :      0.8141                Degrees of Freedom    :         132
Pseudo R-squared    :      0.7313
Spatial Pseudo R-squared:  0.7171
Sigma-square ML     :       0.177                Log likelihood        :     -85.573
S.E of regression   :       0.421                Akaike info criterion :     215.146
                                                 Schwarz criterion     :     281.959

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
-------------------

In [53]:
colname_beta = m2.name_y + '_beta'
df_res[colname_beta] = m2.betas
df_res[colname_beta] = np.round(df_res[colname_beta], 3)
df_res[colname_beta] = df_res[colname_beta].astype(str)

colname_se = m2.name_y + '_se'
df_res[colname_se] = m2.std_err
df_res[colname_se] = np.round(df_res[colname_se], 3)

colname_p = m2.name_y + '_p'
df_res[colname_p] = [t[1] for t in  m2.z_stat]
df_res[colname_p] = np.round(df_res[colname_p], 3)

df_res.loc[df_res[colname_p] <= 0.05, colname_beta] = df_res[colname_beta] + "*"

pr2.append(m2.pr2)
aic.append(m2.aic)
logll.append(m2.logll)

In [54]:
m2 = pysal.model.spreg.ML_Lag(gdf2020[['log_c_trips202006']].values,
                  gdf2020[variable_names20].values,
                  w=wq,
                  name_y='log_c_trips202006',
                  name_x=variable_names)
print(m2.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL LAG (METHOD = FULL)
-----------------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :log_c_trips202006                Number of Observations:         154
Mean dependent var  :      5.2943                Number of Variables   :          22
S.D. dependent var  :      1.0649                Degrees of Freedom    :         132
Pseudo R-squared    :      0.6016
Spatial Pseudo R-squared:  0.6018
Sigma-square ML     :       0.449                Log likelihood        :    -156.894
S.E of regression   :       0.670                Akaike info criterion :     357.787
                                                 Schwarz criterion     :     424.600

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------

In [55]:
colname_beta = m2.name_y + '_beta'
df_res[colname_beta] = m2.betas
df_res[colname_beta] = np.round(df_res[colname_beta], 3)
df_res[colname_beta] = df_res[colname_beta].astype(str)

colname_se = m2.name_y + '_se'
df_res[colname_se] = m2.std_err
df_res[colname_se] = np.round(df_res[colname_se], 3)

colname_p = m2.name_y + '_p'
df_res[colname_p] = [t[1] for t in  m2.z_stat]
df_res[colname_p] = np.round(df_res[colname_p], 3)

df_res.loc[df_res[colname_p] <= 0.05, colname_beta] = df_res[colname_beta] + "*"

pr2.append(m2.pr2)
aic.append(m2.aic)
logll.append(m2.logll)

In [56]:
m2 = pysal.model.spreg.ML_Lag(gdf2021[['log_b_passengers202106']].values,
                  gdf2021[variable_names21].values,
                  w=wq,
                  name_y='log_b_passengers202106',
                  name_x=variable_names)
print(m2.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL LAG (METHOD = FULL)
-----------------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :log_b_passengers202106                Number of Observations:         154
Mean dependent var  :      8.1942                Number of Variables   :          22
S.D. dependent var  :      0.8819                Degrees of Freedom    :         132
Pseudo R-squared    :      0.7509
Spatial Pseudo R-squared:  0.7464
Sigma-square ML     :       0.192                Log likelihood        :     -91.766
S.E of regression   :       0.439                Akaike info criterion :     227.533
                                                 Schwarz criterion     :     294.346

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
-------------------

In [57]:
colname_beta = m2.name_y + '_beta'
df_res[colname_beta] = m2.betas
df_res[colname_beta] = np.round(df_res[colname_beta], 3)
df_res[colname_beta] = df_res[colname_beta].astype(str)

colname_se = m2.name_y + '_se'
df_res[colname_se] = m2.std_err
df_res[colname_se] = np.round(df_res[colname_se], 3)

colname_p = m2.name_y + '_p'
df_res[colname_p] = [t[1] for t in  m2.z_stat]
df_res[colname_p] = np.round(df_res[colname_p], 3)

df_res.loc[df_res[colname_p] <= 0.05, colname_beta] = df_res[colname_beta] + "*"

pr2.append(m2.pr2)
aic.append(m2.aic)
logll.append(m2.logll)

In [58]:
m2 = pysal.model.spreg.ML_Lag(gdf2021[['log_c_trips202106']].values,
                  gdf2021[variable_names21].values,
                  w=wq,
                  name_y='log_c_trips202106',
                  name_x=variable_names)
print(m2.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL LAG (METHOD = FULL)
-----------------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :log_c_trips202106                Number of Observations:         154
Mean dependent var  :      5.1583                Number of Variables   :          22
S.D. dependent var  :      1.1384                Degrees of Freedom    :         132
Pseudo R-squared    :      0.7123
Spatial Pseudo R-squared:  0.7123
Sigma-square ML     :       0.370                Log likelihood        :    -142.098
S.E of regression   :       0.609                Akaike info criterion :     328.196
                                                 Schwarz criterion     :     395.009

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------

In [59]:
colname_beta = m2.name_y + '_beta'
df_res[colname_beta] = m2.betas
df_res[colname_beta] = np.round(df_res[colname_beta], 3)
df_res[colname_beta] = df_res[colname_beta].astype(str)

colname_se = m2.name_y + '_se'
df_res[colname_se] = m2.std_err
df_res[colname_se] = np.round(df_res[colname_se], 3)

colname_p = m2.name_y + '_p'
df_res[colname_p] = [t[1] for t in  m2.z_stat]
df_res[colname_p] = np.round(df_res[colname_p], 3)

df_res.loc[df_res[colname_p] <= 0.05, colname_beta] = df_res[colname_beta] + "*"

pr2.append(m2.pr2)
aic.append(m2.aic)
logll.append(m2.logll)

In [60]:
pr2

[0.7402457913280316,
 0.7324896947476023,
 0.7313435472049021,
 0.6016374182114756,
 0.7509159953682002,
 0.7123364878837465]

In [61]:
aic

[233.91842039932828,
 316.5837175706629,
 215.14579305533488,
 357.78711811191977,
 227.5329590351731,
 328.19639205162514]

In [62]:
logll

[-94.95921019966414,
 -136.29185878533144,
 -85.57289652766744,
 -156.89355905595988,
 -91.76647951758655,
 -142.09819602581257]

### Duration

In [63]:
# df_res = pd.DataFrame()
# df_res['variable'] = m2.name_x

# pr2 = []
# aic = []
# logll = []

In [64]:
m2 = pysal.model.spreg.ML_Lag(gdf2019[['log_c_dur201906']].values,
                  gdf2019[variable_names19].values,
                  w=wq,
                  name_y='log_c_dur201906',
                  name_x=variable_names19)
print(m2.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL LAG (METHOD = FULL)
-----------------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :log_c_dur201906                Number of Observations:         154
Mean dependent var  :      3.0111                Number of Variables   :          22
S.D. dependent var  :      0.5030                Degrees of Freedom    :         132
Pseudo R-squared    :      0.1965
Spatial Pseudo R-squared:  0.1883
Sigma-square ML     :       0.202                Log likelihood        :     -95.478
S.E of regression   :       0.449                Akaike info criterion :     234.955
                                                 Schwarz criterion     :     301.768

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
--------------------------

In [65]:
colname_beta = m2.name_y + '_beta'
df_res[colname_beta] = m2.betas
df_res[colname_beta] = np.round(df_res[colname_beta], 3)
df_res[colname_beta] = df_res[colname_beta].astype(str)

colname_se = m2.name_y + '_se'
df_res[colname_se] = m2.std_err
df_res[colname_se] = np.round(df_res[colname_se], 3)

colname_p = m2.name_y + '_p'
df_res[colname_p] = [t[1] for t in  m2.z_stat]
df_res[colname_p] = np.round(df_res[colname_p], 3)

df_res.loc[df_res[colname_p] <= 0.05, colname_beta] = df_res[colname_beta] + "*"

pr2.append(m2.pr2)
aic.append(m2.aic)
logll.append(m2.logll)

In [66]:
m2 = pysal.model.spreg.ML_Lag(gdf2020[['log_c_dur202006']].values,
                  gdf2020[variable_names20].values,
                  w=wq,
                  name_y='log_c_dur202006',
                  name_x=variable_names19)
print(m2.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL LAG (METHOD = FULL)
-----------------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :log_c_dur202006                Number of Observations:         154
Mean dependent var  :      3.4429                Number of Variables   :          22
S.D. dependent var  :      0.5358                Degrees of Freedom    :         132
Pseudo R-squared    :      0.1679
Spatial Pseudo R-squared:  0.1495
Sigma-square ML     :       0.237                Log likelihood        :    -108.052
S.E of regression   :       0.487                Akaike info criterion :     260.104
                                                 Schwarz criterion     :     326.917

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
--------------------------

In [67]:
colname_beta = m2.name_y + '_beta'
df_res[colname_beta] = m2.betas
df_res[colname_beta] = np.round(df_res[colname_beta], 3)
df_res[colname_beta] = df_res[colname_beta].astype(str)

colname_se = m2.name_y + '_se'
df_res[colname_se] = m2.std_err
df_res[colname_se] = np.round(df_res[colname_se], 3)

colname_p = m2.name_y + '_p'
df_res[colname_p] = [t[1] for t in  m2.z_stat]
df_res[colname_p] = np.round(df_res[colname_p], 3)

df_res.loc[df_res[colname_p] <= 0.05, colname_beta] = df_res[colname_beta] + "*"

pr2.append(m2.pr2)
aic.append(m2.aic)
logll.append(m2.logll)

In [68]:
m2 = pysal.model.spreg.ML_Lag(gdf2021[['log_c_dur202106']].values,
                  gdf2021[variable_names21].values,
                  w=wq,
                  name_y='log_c_dur202106',
                  name_x=variable_names19)
print(m2.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL LAG (METHOD = FULL)
-----------------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :log_c_dur202106                Number of Observations:         154
Mean dependent var  :      3.0848                Number of Variables   :          22
S.D. dependent var  :      0.4975                Degrees of Freedom    :         132
Pseudo R-squared    :      0.1764
Spatial Pseudo R-squared:  0.1600
Sigma-square ML     :       0.202                Log likelihood        :     -95.769
S.E of regression   :       0.450                Akaike info criterion :     235.538
                                                 Schwarz criterion     :     302.351

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
--------------------------

In [69]:
colname_beta = m2.name_y + '_beta'
df_res[colname_beta] = m2.betas
df_res[colname_beta] = np.round(df_res[colname_beta], 3)
df_res[colname_beta] = df_res[colname_beta].astype(str)

colname_se = m2.name_y + '_se'
df_res[colname_se] = m2.std_err
df_res[colname_se] = np.round(df_res[colname_se], 3)

colname_p = m2.name_y + '_p'
df_res[colname_p] = [t[1] for t in  m2.z_stat]
df_res[colname_p] = np.round(df_res[colname_p], 3)

df_res.loc[df_res[colname_p] <= 0.05, colname_beta] = df_res[colname_beta] + "*"

pr2.append(m2.pr2)
aic.append(m2.aic)
logll.append(m2.logll)

In [70]:
m2

In [71]:
df_res = df_res[['variable', 
                 'log_b_passengers201906_beta', 'log_b_passengers201906_se','log_b_passengers201906_p', 
                 'log_b_passengers202006_beta', 'log_b_passengers202006_se', 'log_b_passengers202006_p', 
                 'log_b_passengers202106_beta', 'log_b_passengers202106_se','log_b_passengers202106_p', 
                 'log_c_trips201906_beta', 'log_c_trips201906_se', 'log_c_trips201906_p',
                 'log_c_trips202006_beta','log_c_trips202006_se', 'log_c_trips202006_p',
                 'log_c_trips202106_beta','log_c_trips202106_se', 'log_c_trips202106_p', 
                 'log_c_dur201906_beta','log_c_dur201906_se', 'log_c_dur201906_p', 
                 'log_c_dur202006_beta','log_c_dur202006_se', 'log_c_dur202006_p', 
                 'log_c_dur202106_beta','log_c_dur202106_se', 'log_c_dur202106_p']]

In [72]:
df_res.to_csv('../output/slm_results.csv', sep='\t', index=False)

In [73]:
pr2

[0.7402457913280316,
 0.7324896947476023,
 0.7313435472049021,
 0.6016374182114756,
 0.7509159953682002,
 0.7123364878837465,
 0.1965344153279411,
 0.16790170299554835,
 0.17639376940615953]

In [74]:
logll

[-94.95921019966414,
 -136.29185878533144,
 -85.57289652766744,
 -156.89355905595988,
 -91.76647951758655,
 -142.09819602581257,
 -95.47767752713825,
 -108.05202098649167,
 -95.7688919397295]

In [75]:
aic

[233.91842039932828,
 316.5837175706629,
 215.14579305533488,
 357.78711811191977,
 227.5329590351731,
 328.19639205162514,
 234.9553550542765,
 260.10404197298334,
 235.537783879459]

In [76]:
m2 = pysal.model.spreg.ML_Lag(gdf2020[['log_b_passengers202006']].values,
                  gdf2020[variable_names20].values,
                  w=wq,
                  name_y='log_b_passengers202006',
                  name_x=variable_names20)
print(m2.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL LAG (METHOD = FULL)
-----------------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :log_b_passengers202006                Number of Observations:         154
Mean dependent var  :      7.3406                Number of Variables   :          22
S.D. dependent var  :      0.8141                Degrees of Freedom    :         132
Pseudo R-squared    :      0.7313
Spatial Pseudo R-squared:  0.7171
Sigma-square ML     :       0.177                Log likelihood        :     -85.573
S.E of regression   :       0.421                Akaike info criterion :     215.146
                                                 Schwarz criterion     :     281.959

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
-------------------